In [ ]:
# buildGTI https://gdal.org/drivers/raster/gti.html#raster-gti
# gdaltindex can be used to generate a tile index suitable for use by the GTI driver. There are two possibilities:

# either use directly a vector tile index generated by gdaltindex as the input of the GTI driver

# or generate a small XML .gti wrapper file, for easier use with non-file-based formats such as databases, or for vector formats that do not support setting layer metadata items.

# Formats that support layer metadata are for example GeoPackage (-f GPKG), FlatGeoBuf (-f FlatGeoBuf) or PostGIS (-f PG)

# Setting -tr and -ot is recommended to avoid the GTI driver to have to deduce them by opening the first tile in the index. If the tiles have nodata or mask band, -nodata and -mask should also be set.

# In a GTI context, the extent of all tiles referenced in the tile index must be expressed in a single SRS. Consequently, if input tiles may have different SRS, either -t_srs or -skip_different_projection should be specified.


#more info: https://gdal.org/programs/gdaltindex.html#options-specific-to-use-by-the-gdal-gti-driver and https://github.com/OSGeo/gdal/pull/8983

In [1]:
#!conda update -n base -c conda-forge conda
!conda install -n base --yes --override-channels -c conda-forge mamba 'python_abi=*=*cp*'

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.



In [2]:
!conda create --yes --name gdal_master_env

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /srv/conda/envs/gdal_master_env



Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate gdal_master_env
#
# To deactivate an active environment, use
#
#     $ conda deactivate



In [1]:
!conda activate gdal_master_env


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [6]:
!mamba install --yes -c gdal-master gdal


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['gdal']

conda-forge/linux-64                                        Using cache
conda-forge/noarch

In [3]:
!gdalinfo --version

GDAL 3.7.0, released 2023/05/02


In [5]:
!gdalinfo --version

GDAL 3.7.0, released 2023/05/02


In [ ]:
import adlfs
import glob

In [ ]:
sas_token = "se=2024-02-08T20%3A33Z&sp=racwdl&sv=2022-11-02&sr=c&skoid=598b2582-4d1d-4c4e-9bb2-3ad571b791b5&sktid=f6b6dd5b-f02f-441a-99a0-162ac5060bd2&skt=2024-02-01T20%3A34%3A11Z&ske=2024-02-08T20%3A33%3A00Z&sks=b&skv=2022-11-02&sig=sghHRHkJrGpyT2MFAqUzno5TNsg75gBRKqrqcOZSaTE%3D"
fs = adlfs.AzureBlobFileSystem(account_name="snowmelt", credential=sas_token, anon=True)

In [ ]:
#tile_names

In [ ]:
years = [2015,2016,2017,2018,2019,2020,2021,2022,2023]
years = ['allyears_median','allyears_std','allyears_corr_strength','2015_median','2016_median','2017_median','2018_median','2019_median','2020_median','2021_median','2022_median']

In [ ]:
for year in years:
    f = open(f'gti/file_list/files_{year}.txt','w')
    for fn in fs.glob(f'snowmelt/eric/global/*/*{year}*'):
        f.write(f'https://snowmelt.blob.core.windows.net/{fn}\n')
    f.close()

In [ ]:
!gdalinfo --version

In [ ]:
# add -gti_filename,-tr, -ot, t_srs, [cehck above]

!gdaltindex -t_srs EPSG:4326 tile_index.gti.gpkg --optfile $f'gti/file_list/files_{year}.txt'

In [ ]:
!gdalwarp $f'tile_index.gti.gpkg' $f'/tmp/tif_{year}.tif' -co BIGTIFF=YES -of COG #store in tmp?

In [ ]:
import rioxarray as rxr
ind = rxr.open_rasterio('tile_index.gti.gpkg')

In [ ]:
ind

In [ ]:
for year in years:
    # call gti

In [ ]:
for year in years[5:]:
    if (year == 'allyears_std') | (year == 'allyears_corr_strength') | (year == 'allyears_polyfit'):
        nodata = "nan"
    else:
        nodata = '-32768'
    
    print(f'for year {year}, running gdalwarp...')
    !gdalwarp $f'gti/gti_{year}.gti' $f'/tmp/tif/4326/tif_{year}.tif' -co BIGTIFF=YES -of COG #store in tmp?
    
    print(f'for year {year}, uploading mosaic')
    fs.upload(f'/tmp/tif/4326/tif_{year}.tif',f'/snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_{year}_{resolution}m_4326.tif',overwrite=True)
    
    !rm $f'/tmp/tif/4326/tif_{year}.tif'

In [ ]:
# warp gti to tif